In [1]:
# Import necessary libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

ModuleNotFoundError: No module named 'dash'

In [ ]:
data = pd.read_csv("data_weather_encoded.csv")

In [2]:
data

NameError: name 'data' is not defined

In [11]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pickle
import pandas as pd
import plotly.graph_objs as go
# Create a basic Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("London Temperature Prediction"),
    
    html.Label("Select number of days for prediction:"),
    dcc.Dropdown(
        id='days-dropdown',
        options=[
            {'label': '1 day', 'value': 1},
            {'label': '3 days', 'value': 3},
            {'label': '7 days', 'value': 7},
            {'label': '14 days', 'value': 14},
        ],
        value=1
    ),
    
    html.Label("Select prediction model:"),
    dcc.Dropdown(
        id='model-dropdown',
        options=[
            {'label': 'ANN', 'value': 'ann_future_predictions.pkl'},
            {'label': 'Random Forest', 'value': 'random_forecast_predictions.pkl'},
            {'label': 'ML-RBFNN', 'value': 'ml_rbfnn_future_predictions.pkl'},
            {'label': 'Residual BiLSTM', 'value': 'lstm_future_predictions.pkl'},
            {'label': 'SVR', 'value': 'svr_predictions.pkl'},
        ],
        value='ann_future_predictions.pkl'
    ),
    
    html.Button('Predict', id='predict-button', n_clicks=0),
    
    html.Div(id='output-div')
])


@app.callback(
    Output('output-div', 'children'),
    [Input('predict-button', 'n_clicks')],
    [dash.dependencies.State('days-dropdown', 'value'),
     dash.dependencies.State('model-dropdown', 'value')]
)
def predict_temperature(n_clicks, days, model_filename):
    # Set start date to 2021-01-01
    start_date = "2021-01-01"
    
    # Load model predictions
    with open(model_filename, 'rb') as f:
        predictions = pickle.load(f)
    
    # Handle the specific structures of the three models
    if model_filename == "ann_future_predictions.pkl" or model_filename == "svr_predictions.pkl" or model_filename=="random_forecast_predictions.pkl":
        pred_values_14 = predictions
    elif model_filename == "ml_rbfnn_future_predictions.pkl":
        pred_values_14 = predictions['14_days']
    else:
        pred_values_14 = predictions[14]
        if isinstance(pred_values_14, (list, tuple, np.ndarray)) and len(pred_values_14) == 1:
            pred_values_14 = pred_values_14[0]
    
    # Extract the required prediction based on selected days
    pred_values = pred_values_14[:days]
    
         # Convert start date to datetime and generate date list for prediction
    start_dt = pd.to_datetime(start_date)
    date_list = [(start_dt + pd.Timedelta(days=i)).strftime('%Y-%m-%d') for i in range(0, days)]
    
    # Create bar plot
    bar_plot = dcc.Graph(
        figure={
            'data': [
                go.Bar(
                    x=pred_values,
                    y=date_list,
                    orientation='h',
                    marker={
                        'color': pred_values,
                        'colorscale': [[0, 'cyan'], [1, 'yellow']]
                    }
                )
            ],
            'layout': {
                'title': 'Temperature Predictions',
                'xaxis': {
                    'title': 'Temperature (°C)',
                    'zeroline': True,
                    'zerolinecolor': 'black',
                    'zerolinewidth': 2
                },
                'yaxis': {
                    'title': 'Date',
                    'tickformat': '%Y-%m-%d',  # Ensure only date is displayed
                    'tickvals': date_list  # Explicitly specify which dates to show
                }
            }
        }
    )
    
    return bar_plot




In [12]:
if __name__ == '__main__':
    app.run_server(debug=True)
